In [1]:
import dask.dataframe as dd
from dask.distributed import Client

#Initializing client
client = Client()
client

/home/user/anaconda3/envs/overscripted/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/user/anaconda3/envs/overscripted/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


Client Scheduler: tcp://127.0.0.1:42879 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 4.14 GB


In [6]:
#read sample data
df = dd.read_parquet('../../databases/clean_unindexed.parquet', engine='pyarrow', columns=['value_1000', 'value_len'])

Index(['value_1000', 'value_len'], dtype='object')


In [8]:
#printing some data so could have an overview of what len to expect
dfmean = df['value_len'].mean()
dfmin = df['value_len'].min()
dfmax = df['value_len'].max()
dfstd = df['value_len'].std()
mc = dd.compute(dfmean, dfmin, dfmax, dfstd);
print(mc)

(1313.8734164674954, 0, 4496969, 25831.732898438877)


In [39]:
import tldextract
def extract_domain(url):
    try:
        extracted = tldextract.extract(url)
        return '{}.{}'.format(extracted.domain, extracted.suffix)
    except Exception as e:
        return 'ERROR'

In [59]:
#Get very simplified value domain to have a very simple overview of the domain unique count
#Doing like this does not mean all the value field is equal, just the begining, 
#but for the propouse of this investigation this is good enough
#and processing just the "domain" is pretty much all I can do with the processing power I have at my disposal
df['value_domain'] = df['value_1000'].apply(extract_domain)


In [ ]:
#Save to parquet the processed values with the simplified domain column
#so next time I can read from file instead of waiting very long time to process this column 

df.to_parquet('df_value1000_domain.parquet')

In [2]:
#Read last saved data (I can start analysing from here instead of doing all previous steps)
hdf = dd.read_parquet('df_value1000_domain.parquet', engine='pyarrow')
hdf.head()

,value_1000,value_len,value_domain
0,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko...,68,Mozilla.
1,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko...,68,Mozilla.
2,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko...,68,Mozilla.
3,Netscape,8,Netscape.
4,5.0 (X11),9,5.0 (X11).


In [4]:
#Get all rows that have the value_len above 2000 characters,
#since I want to identify what the really large values are
#I really dont have to worry about the small values and I can eliminate them here

#Also, I'm not sure what "really large values" refers to, I'm just guesing that for sure its not something this small
f = hdf[hdf['value_len'] > 2000]

In [69]:
#getting to know how many rows I'm dealing with now
len(f)

(value_1000      113790686
 value_len       113790686
 value_domain    113790686
 dtype: int64, value_1000      7214922
 value_len       7214922
 value_domain    7214922
 dtype: int64)

In [72]:
#getting to know how many unique domains I'm dealing with
c = f['value_domain'].value_counts().compute()
c.head(10)

834540.                              144483
{"ScribeTransport".                  119637
{"insdrSV".                          116564
{"criteo_pt_cdb_metrics_expires".     91008
3rlQ9p29SeCbCmQ934fYVA$0.             89735
H8wotBY9TFu5Q4Y3_iSQng$0.             77243
Rn8kD3YeRdCYuK-Ya7fNyw$0.             77085
{"ins-today-sId".                     73139
Na9BL8mAQgqyMAy1zxOlJg$0.             69490
Ytag3XR0TQWbMYl0wfPz1Q$0.             66683
Name: value_domain, dtype: int64

In [76]:
#Again saving data on disk so I can access it when I want with no need to 
#reprocess things since it takes too much time with the processing power I have

# c.to_csv('value_domain_count.csv')

/home/user/anaconda3/envs/overscripted/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [5]:
#grouping by domain 
group = f.groupby('value_domain')

In [6]:
#Evaluating the mean, std, min, max and count for the value_len for each unique domain
result = group.agg({'value_len': ['mean', 'std', 'min', 'max', 'count']})
computed_result = result.compute()

In [8]:
computed_result.head(10)

value_len  \
                                                             mean   
value_domain                                                        
 🏳️‍🌈\",\"protected\".                                2020.571429   
!xticcsep*!!xticcsep*!!xticcsep*!0","lscache-.       41685.000000   
!xticcsep*!!xticcsep*!!xticcsep*!{\"version\".       41994.000000   
"BODY.                                                2384.000000   
%22%2C%22ct%22%3A%22Mountain%20View%22%2C%22co%...    2078.253731   
%2520ADFS%253F%2520%257C%2520Okta%26_biz_n%3D0%...    2103.000000   
%2520ADFS%253F%2520%257C%2520Okta%26_biz_n%3D1%...    2555.000000   
%252520El%252520Otomobiller%252520%25253A%25252...    2482.114286   
%3A1366","_dyfs".                                    24857.266667   
%3Ahttp%3A.                                         283397.000000   

                                                                         \
                                                            std     min   
value_domain                                                              
 🏳️‍🌈\",\"protected\".                                17.386366    2014   
!xticcsep*!!xticcsep*!!xticcsep*!0","lscache-.         0.000000   41685   
!xticcsep*!!xticcsep*!!xticcsep*!{\"version\".      1995.605283   38750   
"BODY.                                                 0.000000    2384   
%22%2C%22ct%22%3A%22Mountain%20View%22%2C%22co%...    70.801974    2039   
%2520ADFS%253F%2520%257C%2520Okta%26_biz_n%3D0%...     0.000000    2103   
%2520ADFS%253F%2520%257C%2520Okta%26_biz_n%3D1%...     0.000000    2555   
%252520El%252520Otomobiller%252520%25253A%25252...    92.420269    2347   
%3A1366","_dyfs".                                    260.844088   24540   
%3Ahttp%3A.                                            0.000000  283397   

                                                                  
                                                       max count  
value_domain                                                      
 🏳️‍🌈\",\"protected\".                                2060     7  
!xticcsep*!!xticcsep*!!xticcsep*!0","lscache-.       41685     2  
!xticcsep*!!xticcsep*!!xticcsep*!{\"version\".       44639    10  
"BODY.                                                2384     6  
%22%2C%22ct%22%3A%22Mountain%20View%22%2C%22co%...    2246    67  
%2520ADFS%253F%2520%257C%2520Okta%26_biz_n%3D0%...    2103     2  
%2520ADFS%253F%2520%257C%2520Okta%26_biz_n%3D1%...    2555     6  
%252520El%252520Otomobiller%252520%25253A%25252...    2571   105  
%3A1366","_dyfs".                                    25287    15  
%3Ahttp%3A.                                         283397     2

In [31]:
#again saving result to eliminate the need of recomputing
computed_result['value_len'].to_parquet('computed_result.parquet')

/home/user/anaconda3/envs/overscripted/lib/python3.6/site-packages/pyarrow/pandas_compat.py:114: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  result = infer_dtype(pandas_collection)


In [ ]:
#read computed results if needed
# computed_result = dd.read_parquet('computed_result.parquet', engine='pyarrow')

In [33]:
#I noticed that some domains have very little occurencies
#so I decided to analyse the ones that appear the most, 
#or at least a number of times that its interesting to analyse
cr1 = computed_result[computed_result.value_len['count'] > 500]

In [34]:
#I was still left with too many rows,
#so I decided to filter little bit more by value_len 
#(again, the objective here is to know what is in the really large values, 
#so I dont think filtering the smaller ones is a problem, and it will make process so much faster)
crf = cr1[cr1.value_len['max'] > 5000]

In [35]:
crf.head(10)

value_len                                 \
                                       mean            std     min     max   
value_domain                                                                 
0.                              6162.615723   28838.128106    2151  452979   
1.                              6357.259101   23466.783412    2001  165355   
12_20171214163401_203aad7a".  137528.594893      33.943976  137448  137611   
12_20171214163401_22fdafb1".  193743.342228      15.341908  193679  193747   
12_20171214163401_2ee09a0c".  104294.223448   10516.059642  103878  436505   
12_20171214163401_4ab79343".  193679.000000       0.000000  193679  193679   
12_20171214163401_4f925001".  137545.389857      29.873971  137448  137693   
1438".                        197677.328664  163455.948478    2884  783332   
1439".                        162237.547330  149440.117553    2344  492106   
195_af_lpdid".                  3136.151181    1389.901671    2286    6445   

                                     
                              count  
value_domain                         
0.                             2048  
1.                              934  
12_20171214163401_203aad7a".   1723  
12_20171214163401_22fdafb1".   9444  
12_20171214163401_2ee09a0c".  17127  
12_20171214163401_4ab79343".   1236  
12_20171214163401_4f925001".    631  
1438".                          928  
1439".                          824  
195_af_lpdid".                  635

In [36]:
#By this point I'm left with 220 unique domains, with I can actually analyse now
len(crf)

220

In [4]:
#I'm still not very familiar with neither dask or pandas, but I find pandas easier to use
#since I got a smaller dataset to analyse I'll use pandas for couple analyses from here on out
import pandas as pn
cc = pn.read_parquet('computed_result.parquet')

/home/user/anaconda3/envs/overscripted/lib/python3.6/site-packages/pyarrow/pandas_compat.py:698: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/home/user/anaconda3/envs/overscripted/lib/python3.6/site-packages/pyarrow/pandas_compat.py:725: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/home/user/anaconda3/envs/overscripted/lib/python3.6/site-packages/pyarrow/pandas_compat.py:742: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [20]:
#The top 5 occurencies
pn.options.display.float_format = '{:.2f}'.format
formated = cc.sort_values('count', ascending=False).head(10)
formated

,mean,std,min,max,count
value_domain,,,,,
"{""ScribeTransport"".",4128.59,1406.46,2001,7211,93409
"{""ins-today-sId"".",5037.69,14446.52,2002,87748,60426
"{""criteo_pt_cdb_metrics_expires"".",9529.66,53326.72,2003,692032,47543
font-face{font-family.,162363.28,172503.75,2634,648067,45059
"{""CLOUDFLARE.",514484.07,634151.12,4356,3253324,42660
"{""__qubitUACategorisation"".",64927.71,105887.48,2018,368966,40003
Na9BL8mAQgqyMAy1zxOlJg$0.,2236.68,178.84,2001,3312,37945
935971.,3726.06,396.41,3248,4695,33010
"{""insdrSV"".",4026.30,12823.05,2002,191041,32981
